<a href="https://colab.research.google.com/github/YellaCRE/FIFA_RecommandSystem/blob/main/FIFA_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#피파온라인4 컨텐츠기반 선수추천알고리즘_EDA

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/2022.2 피파선수 추천 프로젝트/player_df.csv")

#간단한 통계 수치 및 전처리

In [ ]:
data[['스탯']].head()

,스탯
0,101
1,101
2,105
3,102
4,105


In [ ]:
data.groupby('포지션').size().sort_values(ascending=False)

포지션
CB     8723
ST     7862
CM     6392
GK     5630
CDM    3924
LB     3402
RB     3383
CAM    3189
RM     2630
LM     2604
RW     1456
LW     1387
CF      537
RWB     322
LWB     299
LF        2
RF        1
dtype: int64

In [ ]:
data.groupby('name').size().sort_values(ascending=False).head()

name
크리스티아누 호날두    30
케빈 더브라위너      27
마르셀루          27
리오넬 메시        27
카림 벤제마        26
dtype: int64

In [ ]:
data.sort_values(by = ['스탯'],ascending=False)

,index,id,name,시즌,포지션,스탯,키,몸무게,속력,가속력,...,도움,유효슈팅,일반슈팅,패스성공률,드리블성공률,공중볼경합성공률,태클성공률,차단성공률,선방,평점
23,23,101001397,지네딘 지단,ICON,CAM,109,185cm,77kg,105,108,...,0.2(350),0.5(685),0.2(227),"92.7%17,360/18,719","91.1%15,364/16,867","5.9%101/1,700","68.1%829/1,218",7.1%43/608,0.2(313),6.8
110,110,101254642,페렌츠 푸스카스,ICON,CF,109,172cm,72kg,112,111,...,0.2(52),0.8(174),0.2(35),"92.6%2,040/2,204","85.6%1,813/2,119",3.5%7/200,61.1%77/126,0%0/42,0.1(11),7.0
87,87,101190045,요한 크루이프,ICON,CF,109,180cm,71kg,104,110,...,0.3(168),0.6(366),0.2(111),"89.2%6,507/7,292","88.4%5,857/6,629",4.1%26/637,63.7%262/411,3.6%6/165,0.1(83),6.8
62,62,101037576,호나우두,ICON,ST,109,183cm,78kg,116,116,...,0.4(48),1.1(155),0.4(49),"92.5%1,242/1,342","85.4%918/1,075",5.7%9/157,62.3%33/53,0%0/21,0.1(10),7.3
102,102,101238380,레프 야신,ICON,GK,108,189cm,82kg,72,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23153,23153,300261667,M. 오브라이언,LIVE,RM,39,176cm,71kg,52,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45882,45882,320260623,M. 라이트,20,LB,39,178cm,81kg,42,43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27708,27708,317235352,토미 케세모델,17,CM,39,173cm,75kg,20,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34005,34005,318245344,케빈 카탈란,18,GK,39,178cm,69kg,13,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.sort_values(by = ['속력','가속력'],ascending=False).head()

,index,id,name,시즌,포지션,스탯,키,몸무게,속력,가속력,...,도움,유효슈팅,일반슈팅,패스성공률,드리블성공률,공중볼경합성공률,태클성공률,차단성공률,선방,평점
27,27,101001625,티에리 앙리,ICON,ST,107,188cm,83kg,119,117,...,0.3(226),1.0(878),0.3(226),"92.6%8,107/8,754","86.2%6,795/7,885",7.7%77/995,59.1%298/504,3.6%5/138,0.1(70),7.1
62,62,101037576,호나우두,ICON,ST,109,183cm,78kg,116,116,...,0.4(48),1.1(155),0.4(49),"92.5%1,242/1,342","85.4%918/1,075",5.7%9/157,62.3%33/53,0%0/21,0.1(10),7.3
52,52,101009676,사뮈엘 에토,ICON,ST,107,179cm,75kg,116,115,...,0.3(64),1.1(274),0.3(68),"91%2,573/2,826","87.3%2,265/2,595",5.7%11/192,55.6%74/133,8.1%3/37,0.0(8),7.2
76,76,101138449,카카,ICON,CAM,107,186cm,78kg,116,115,...,0.3(436),"0.8(1,079)",0.2(300),"91.2%14,091/15,448","87%13,149/15,113","5.5%81/1,482",60.5%576/952,6%19/318,0.1(148),6.9
106,106,101247553,가린샤,ICON,RW,108,169cm,72kg,116,113,...,0.3(24),0.7(55),0.2(13),90.6%706/779,85.1%692/813,10.9%7/64,66.7%30/45,0%0/20,0.1(6),6.9


In [ ]:
data.sort_values(by = ['스탯','평점'],ascending=False).head()

,index,id,name,시즌,포지션,스탯,키,몸무게,속력,가속력,...,도움,유효슈팅,일반슈팅,패스성공률,드리블성공률,공중볼경합성공률,태클성공률,차단성공률,선방,평점
62,62,101037576,호나우두,ICON,ST,109,183cm,78kg,116,116,...,0.4(48),1.1(155),0.4(49),"92.5%1,242/1,342","85.4%918/1,075",5.7%9/157,62.3%33/53,0%0/21,0.1(10),7.3
110,110,101254642,페렌츠 푸스카스,ICON,CF,109,172cm,72kg,112,111,...,0.2(52),0.8(174),0.2(35),"92.6%2,040/2,204","85.6%1,813/2,119",3.5%7/200,61.1%77/126,0%0/42,0.1(11),7.0
23,23,101001397,지네딘 지단,ICON,CAM,109,185cm,77kg,105,108,...,0.2(350),0.5(685),0.2(227),"92.7%17,360/18,719","91.1%15,364/16,867","5.9%101/1,700","68.1%829/1,218",7.1%43/608,0.2(313),6.8
87,87,101190045,요한 크루이프,ICON,CF,109,180cm,71kg,104,110,...,0.3(168),0.6(366),0.2(111),"89.2%6,507/7,292","88.4%5,857/6,629",4.1%26/637,63.7%262/411,3.6%6/165,0.1(83),6.8
103,103,101242519,에우제비우,ICON,CF,108,175cm,76kg,115,112,...,0.3(745),"1.0(2,581)",0.3(664),"91.6%24,956/27,233","86.2%20,866/24,200","6.2%164/2,628","53.7%668/1,244",2%9/446,0.1(139),7.2


#강화효과

아이콘 지단 
1강 112

2강 113 +1
3강 114 +2
4강 116 +4

5강 118 +6
6강 120 +8
7강 123 +11

8강 127 +15
9강 131 +19
10강 136 +24

LH 손흥민
1강 91

2강 92 +1
3강 93 +2
4강 95 +4

5강 97 +6
6강 99 +8
7강 102 +11

8강 106 +15

현실적인 강화수치인 금카 8강을 기준으로 최대 15의 증가 효과가 있음



#랭커선수
랭킹 6위 Visual유곽구 21 테오 에르난데스 8강 사용 30경기 출전(1강 오버롤 75)
랭킹 9위 KDF강준호 K18 홍철 7강 사용 30경기 출전(1강 오버롤 77)
랭킹 62위 HyeinLee 21 알폰소 데이비스 8강 사용 30경기 출전(1강 오버롤 73)

오버롤 1강 기준 70이하의 선수는 체감을 떠나 스탯에서 많이 딸림을 알 수 있음

= 0강 기준 60이하의 선수는 데이터군에서 제외

#데이터 세분화

In [ ]:
data = data[data.스탯 >= 60]

In [ ]:
data.describe()

,스탯,ID,GK공격가담,GK능숙한펀칭,GK멀리던지기,GK소극적크로스수비,GK적극적크로스수비,GK침착한1:1수비,강철몸,개인플레이선호,...,침착성,GK 다이빙,GK 핸들링,GK 킥,GK 반응속도,GK 위치선정,개인기,급여,left,right
count,21953.000000,2.195300e+04,21953.000000,21953.000000,21953.000000,21953.000000,21953.000000,21953.000000,21953.000000,21953.000000,...,21953.000000,21953.000000,21953.000000,21953.000000,21953.000000,21953.000000,21953.000000,21953.000000,21953.000000,21953.000000
mean,74.423769,3.276726e+08,0.002551,0.008518,0.018084,0.011570,0.023687,0.005831,0.058489,0.037626,...,67.100214,18.211862,18.017264,17.911903,18.396256,18.136747,2.698082,10.180021,3.647565,4.514736
std,12.160553,1.050363e+08,0.050443,0.091902,0.133259,0.106943,0.152075,0.076137,0.234670,0.190294,...,17.274178,19.119074,18.487074,17.822957,19.480598,18.770110,0.952232,5.566605,0.994432,0.937591
min,60.000000,1.010000e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,11.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,1.000000,1.000000
25%,63.000000,2.462198e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,55.000000,8.000000,8.000000,8.000000,8.000000,8.000000,2.000000,5.000000,3.000000,5.000000
50%,73.000000,3.002264e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,66.000000,12.000000,12.000000,12.000000,12.000000,12.000000,3.000000,9.000000,3.000000,5.000000
75%,83.000000,3.202251e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,80.000000,18.000000,18.000000,18.000000,18.000000,18.000000,3.000000,15.000000,5.000000,5.000000
max,109.000000,5.081930e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,117.000000,109.000000,108.000000,106.000000,112.000000,111.000000,5.000000,27.000000,5.000000,5.000000


포지션별로 데이터 세분화

포지션별로 필요없는 능력치를 제거하고 코사인유사도 돌려보기

In [ ]:
data_GK = data[data.포지션 == 'GK']
isB = data['포지션'].str.contains("B") #수비수
isM = data['포지션'].str.contains("M") #미드필더
isF = data['포지션'].str.contains("F|ST|RW|LW") #공격수 #다음코드에는 LWB와 RWB도 포함되지만 윙백의 경우도 윙으로 사용이 가능하기 때문에 허용 

data_B = data[isB]
data_M = data[isM]
data_F = data[isF]

print(data_GK.head())
print(data_B.head())
print(data_M.head())
print(data_F.head())

   포지션   스탯         ID       이름  GK공격가담  GK능숙한펀칭  GK멀리던지기  GK소극적크로스수비  \
0   GK  101  101000001  데이비드 시먼     0.0      1.0      1.0         0.0   
9   GK  103  101000805    옌스 레만     0.0      0.0      1.0         0.0   
40  GK  107  101005479     카시야스     0.0      1.0      1.0         0.0   
57  GK  101  101020170  비토르 바이아     0.0      1.0      0.0         0.0   
67  GK  105  101048940   페트르 체흐     0.0      0.0      1.0         1.0   

    GK적극적크로스수비  GK침착한1:1수비  ...  침착성  GK 다이빙  GK 핸들링  GK 킥  GK 반응속도  GK 위치선정  \
0          0.0         0.0  ...   86      98     105   106      101      103   
9          1.0         0.0  ...   88     102     106   101      101      105   
40         0.0         1.0  ...   98     109     108    98      109      109   
57         0.0         0.0  ...   84     104     103    95      102       99   
67         0.0         1.0  ...   92     105     107    98      110      108   

    개인기  급여  left  right  
0     1  20     3      5  
9     1  21     3      5  

#포지션 별 주요스탯구분

default

['키', '몸무게', '속력', '가속력', '골 결정력',
       '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '페널티 킥', '짧은 패스', '시야', '크로스', '긴 패스',
       '프리킥', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '대인 수비', '태클',
       '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프', '침착성', 'GK 다이빙',
       'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정', '개인기', '급여', 'left', 'right','체형'
       ]

In [ ]:
data_GK_num = data_GK[['키', '몸무게', '속력', '가속력', '골 결정력',
       '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '페널티 킥', '짧은 패스', '시야', '크로스', '긴 패스',
       '프리킥', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '대인 수비', '태클',
       '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프', '침착성', 'GK 다이빙',
       'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정', '개인기', '급여', 'left', 'right',
       ]]
data_B_num = data_B[['키', '몸무게', '속력', '가속력', '골 결정력',
       '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '페널티 킥', '짧은 패스', '시야', '크로스', '긴 패스',
       '프리킥', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '대인 수비', '태클',
       '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프', '침착성', 'GK 다이빙',
       'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정', '개인기', '급여', 'left', 'right',
       ]]
data_M_num = data_M[['키', '몸무게', '속력', '가속력', '골 결정력',
       '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '페널티 킥', '짧은 패스', '시야', '크로스', '긴 패스',
       '프리킥', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '대인 수비', '태클',
       '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프', '침착성', 'GK 다이빙',
       'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정', '개인기', '급여', 'left', 'right',
       ]]
data_F_num = data_F[['키', '몸무게', '속력', '가속력', '골 결정력',
       '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '페널티 킥', '짧은 패스', '시야', '크로스', '긴 패스',
       '프리킥', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '대인 수비', '태클',
       '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프', '침착성', 'GK 다이빙',
       'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정', '개인기', '급여', 'left', 'right',
       ]]

print(data_GK_num.head())
print(data_B_num.head())
print(data_M_num.head())
print(data_F_num.head())

      키  몸무게  속력  가속력  골 결정력  슛 파워  중거리 슛  위치 선정  발리 슛  페널티 킥  ...  침착성  \
0   193   93  54   70     48    45     39     35    45     50  ...   86   
9   190   87  75   76     35    58     35     36    16     43  ...   88   
40  185   80  85   86     40    56     41     44    42     52  ...   98   
57  184   78  66   74     43    51     36     44    44     54  ...   84   
67  196   90  62   67     34    53     30     40    19     38  ...   92   

    GK 다이빙  GK 핸들링  GK 킥  GK 반응속도  GK 위치선정  개인기  급여  left  right  
0       98     105   106      101      103    1  20     3      5  
9      102     106   101      101      105    1  21     3      5  
40     109     108    98      109      109    1  22     5      3  
57     104     103    95      102       99    1  20     3      5  
67     105     107    98      110      108    1  22     5      3  

[5 rows x 40 columns]
      키  몸무게   속력  가속력  골 결정력  슛 파워  중거리 슛  위치 선정  발리 슛  페널티 킥  ...  침착성  \
3   179   79  102  100     66    88     56     7

In [ ]:
cos_GK = cosine_similarity(data_GK_num, data_GK_num)
cos_GK

array([[1.        , 0.98096728, 0.98976019, ..., 0.86451052, 0.87170073,
        0.86648399],
       [0.98096728, 1.        , 0.98424583, ..., 0.84349886, 0.85979862,
        0.84348467],
       [0.98976019, 0.98424583, 1.        , ..., 0.87273753, 0.87408465,
        0.86946654],
       ...,
       [0.86451052, 0.84349886, 0.87273753, ..., 1.        , 0.98019592,
        0.99528372],
       [0.87170073, 0.85979862, 0.87408465, ..., 0.98019592, 1.        ,
        0.98119373],
       [0.86648399, 0.84348467, 0.86946654, ..., 0.99528372, 0.98119373,
        1.        ]])

In [ ]:
cos_B = cosine_similarity(data_B_num, data_B_num)
cos_B

array([[1.        , 0.98975433, 0.98362087, ..., 0.9811238 , 0.98641323,
        0.99131689],
       [0.98975433, 1.        , 0.9686209 , ..., 0.98651386, 0.97746008,
        0.98665134],
       [0.98362087, 0.9686209 , 1.        , ..., 0.96167181, 0.98487705,
        0.98041004],
       ...,
       [0.9811238 , 0.98651386, 0.96167181, ..., 1.        , 0.98404537,
        0.98694949],
       [0.98641323, 0.97746008, 0.98487705, ..., 0.98404537, 1.        ,
        0.99383079],
       [0.99131689, 0.98665134, 0.98041004, ..., 0.98694949, 0.99383079,
        1.        ]])

In [ ]:
cos_M = cosine_similarity(data_M_num, data_M_num)
cos_M

array([[1.        , 0.98914146, 0.99338692, ..., 0.97426815, 0.7749318 ,
        0.98929924],
       [0.98914146, 1.        , 0.99645446, ..., 0.9873565 , 0.7647271 ,
        0.98590458],
       [0.99338692, 0.99645446, 1.        , ..., 0.98307689, 0.75836029,
        0.98677781],
       ...,
       [0.97426815, 0.9873565 , 0.98307689, ..., 1.        , 0.8216645 ,
        0.98570395],
       [0.7749318 , 0.7647271 , 0.75836029, ..., 0.8216645 , 1.        ,
        0.81810981],
       [0.98929924, 0.98590458, 0.98677781, ..., 0.98570395, 0.81810981,
        1.        ]])

In [ ]:
cos_F = cosine_similarity(data_F_num, data_F_num)
cos_F

array([[1.        , 0.99546119, 0.99605487, ..., 0.97612036, 0.98810424,
        0.98178047],
       [0.99546119, 1.        , 0.99789447, ..., 0.97308449, 0.98731984,
        0.97719892],
       [0.99605487, 0.99789447, 1.        , ..., 0.97222015, 0.98574319,
        0.97589197],
       ...,
       [0.97612036, 0.97308449, 0.97222015, ..., 1.        , 0.98249496,
        0.96879476],
       [0.98810424, 0.98731984, 0.98574319, ..., 0.98249496, 1.        ,
        0.99311555],
       [0.98178047, 0.97719892, 0.97589197, ..., 0.96879476, 0.99311555,
        1.        ]])

#공격수
https://www.thisisgame.com/tinyagit/nboard/199/?n=39532
https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=445566&logNo=221579062673


공격수 공통 중요 능력치 
 - 골 결정력, 슛 파워, 중거리 슛, 속력, 가속력, 반응 속도, 짧은 패스, 볼 컨트롤, 드리블 (헤더,몸싸움,점프,침착성) 왼발,오른발 가산점

스트라이커 
- 공통 중요 능력치 외 발리슛, 헤딩, 몸싸움

중앙 공격수 
- 공통 중요 능력치 외 발리슛, 헤딩, 몸싸움, 크로스

측면 공격수 
- 공통 중요 능력치 외 민첩성, 스테미너, 긴 패스, 크로스, 시야

In [ ]:
isST = data_F['포지션'].str.contains("ST") #스트라이커
isCF = data_F['포지션'].str.contains("F") #중앙공격수
isW = data_F['포지션'].str.contains("LW|RW") #측면공격수

data_ST = data_F[isST]
data_CF = data_F[isCF]
data_W = data_F[isW]

print(data_ST.head())
print(data_CF.head())
print(data_W.head())

   포지션   스탯         ID         이름  GK공격가담  GK능숙한펀칭  GK멀리던지기  GK소극적크로스수비  \
1   ST  101  101000195     로비 파울러     0.0      0.0      0.0         0.0   
7   ST  101  101000393     테디 셰링엄     0.0      0.0      0.0         0.0   
10  ST  104  101000942  크리스티안 비에리     0.0      0.0      0.0         0.0   
20  ST  105  101001198    필리포 인자기     0.0      0.0      0.0         0.0   
27  ST  107  101001625     티에리 앙리     0.0      0.0      0.0         0.0   

    GK적극적크로스수비  GK침착한1:1수비  ...  침착성  GK 다이빙  GK 핸들링  GK 킥  GK 반응속도  GK 위치선정  \
1          0.0         0.0  ...  101      15      19    16       18       18   
7          0.0         0.0  ...   99      19      17    13       13       18   
10         0.0         0.0  ...  105      18      15    19       14       24   
20         0.0         0.0  ...  109      26      25    25       14       26   
27         0.0         0.0  ...  115      20      22    17       22       20   

    개인기  급여  left  right  
1     4  23     5      3  
7     4  23   

In [ ]:
data_ST_num = data_ST[['키', '몸무게', '속력', '가속력', '골 결정력',
       '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '짧은 패스', '시야',
       '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '헤더', '몸싸움', 
       '적극성', '점프', '침착성', '개인기', '급여', 'left', 'right',
       ]]
data_CF_num = data_CF[['키', '몸무게', '속력', '가속력', '골 결정력',
       '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '짧은 패스', '시야', 
       '크로스', '긴 패스', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '헤더', '몸싸움', '스태미너', 
       '적극성', '점프', '침착성', '개인기', '급여', 'left', 'right',
       ]]
data_W_num = data_W[['키', '몸무게', '속력', '가속력',
       '중거리 슛', '시야', '크로스', '긴 패스',
       '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도',
       '몸싸움', '스태미너', '적극성', '침착성', '개인기', '급여', 'left', 'right',
       ]]

print(data_ST_num.head())
print(data_CF_num.head())
print(data_W_num.head())

      키  몸무게   속력  가속력  골 결정력  슛 파워  중거리 슛  위치 선정  발리 슛  짧은 패스  ...  반응 속도  \
1   175   72  101  104    104   104    101    105    97     91  ...    103   
7   185   79   89   88    104   105    102    107   105     94  ...    105   
10  185   82   97  100    109   106    105    109   101     96  ...    105   
20  181   74  103  108    113   104    101    118   102     95  ...    113   
27  188   83  119  117    115   106    109    108   106    101  ...    107   

     헤더  몸싸움  적극성   점프  침착성  개인기  급여  left  right  
1   102   95   89   97  101    4  23     5      3  
7   106   99   92   99   99    4  23     3      5  
10  109  111  105  103  105    3  24     5      4  
20  105   91   80  106  109    3  25     3      5  
27   98   96   90   94  115    5  26     4      5  

[5 rows x 25 columns]
      키  몸무게   속력  가속력  골 결정력  슛 파워  중거리 슛  위치 선정  발리 슛  짧은 패스  ...   헤더  몸싸움  \
14  174   74  107  105    107   105    109    107    97    108  ...   95   90   
21  168   67   98  108    104   10

In [ ]:
cos_ST = cosine_similarity(data_ST_num, data_ST_num)
cos_ST

array([[1.        , 0.99652501, 0.99788554, ..., 0.98738188, 0.99092795,
        0.99074734],
       [0.99652501, 1.        , 0.99852144, ..., 0.97947843, 0.98600404,
        0.99068149],
       [0.99788554, 0.99852144, 1.        , ..., 0.9820788 , 0.9869009 ,
        0.98961894],
       ...,
       [0.98738188, 0.97947843, 0.9820788 , ..., 1.        , 0.99655078,
        0.99252923],
       [0.99092795, 0.98600404, 0.9869009 , ..., 0.99655078, 1.        ,
        0.99681407],
       [0.99074734, 0.99068149, 0.98961894, ..., 0.99252923, 0.99681407,
        1.        ]])

In [ ]:
cos_CF = cosine_similarity(data_CF_num, data_CF_num)
cos_CF

array([[1.        , 0.99826947, 0.99851561, ..., 0.94978365, 0.97289003,
        0.99374841],
       [0.99826947, 1.        , 0.99678773, ..., 0.94005198, 0.97097371,
        0.99127393],
       [0.99851561, 0.99678773, 1.        , ..., 0.95750796, 0.9766387 ,
        0.99489534],
       ...,
       [0.94978365, 0.94005198, 0.95750796, ..., 1.        , 0.98090578,
        0.97172177],
       [0.97289003, 0.97097371, 0.9766387 , ..., 0.98090578, 1.        ,
        0.99018853],
       [0.99374841, 0.99127393, 0.99489534, ..., 0.97172177, 0.99018853,
        1.        ]])

In [ ]:
cos_W = cosine_similarity(data_W_num, data_W_num)
cos_W

array([[1.        , 0.99746038, 0.99847982, ..., 0.99620054, 0.98785054,
        0.98382575],
       [0.99746038, 1.        , 0.99806478, ..., 0.99564019, 0.98639524,
        0.98438298],
       [0.99847982, 0.99806478, 1.        , ..., 0.99522   , 0.98643868,
        0.98279937],
       ...,
       [0.99620054, 0.99564019, 0.99522   , ..., 1.        , 0.99407706,
        0.98760557],
       [0.98785054, 0.98639524, 0.98643868, ..., 0.99407706, 1.        ,
        0.98719815],
       [0.98382575, 0.98438298, 0.98279937, ..., 0.98760557, 0.98719815,
        1.        ]])

#미드필더

미드필더 공통 중요 능력치
- 위치 선정, 속력, 반응 속도, 짧은 패스, 긴 패스, 볼 컨트롤, 시야

속가, 슛파워, 중거리, 짧패, 시야, 긴패스, 드리, 볼컨, 민첩, 밸런, 몸싸움, 스태미너, 점프, 침착성
수비스탯에 따라 두가지로 나눈다
반응 태클 가로채기 헤더 

중앙 미드필더
- 공통 중요 능력치 외 중거리 슛, 헤딩, 스테미너, 몸싸움, 드리블, 태클

공격형 미드필더
- 골 결정력, 슛 파워, 중거리 슛, 헤딩, 드리블

수비형 미드필더
- 공통 중요 능력치 외 헤딩, 스테미너, 몸싸움, 태클, 슬라이딩 태클, 대인 수비, 적극성

측면 미드필더
- 공통 중요 능력치 외 중거리 슛, 가속력, 민첩성, 스테미너, 크로스, 드리블

In [ ]:
isCM = data_M['포지션'].str.contains("CM|CDM") #중미
isCAM = data_M['포지션'].str.contains("CAM") #공미
isWM = data_M['포지션'].str.contains("LM|RM") #윙미드필더

data_CM = data_M[isCM]
data_CAM = data_M[isCAM]
data_WM = data_M[isWM]

print(data_CM.head())
print(data_CAM.head())
print(data_WM.head())

    포지션   스탯         ID         이름  GK공격가담  GK능숙한펀칭  GK멀리던지기  GK소극적크로스수비  \
2   CDM  105  101000240       로이 킨     0.0      0.0      0.0         0.0   
4    CM  105  101000246      폴 스콜스     0.0      0.0      0.0         0.0   
24  CDM  106  101001419  파트리크 비에이라     0.0      0.0      0.0         0.0   
26  CDM  104  101001620    에마뉘엘 프티     0.0      0.0      0.0         0.0   
28  CDM  105  101001668   클로드 마켈렐레     0.0      0.0      0.0         0.0   

    GK적극적크로스수비  GK침착한1:1수비  ...  침착성  GK 다이빙  GK 핸들링  GK 킥  GK 반응속도  GK 위치선정  \
2          0.0         0.0  ...  100      22      25    17       17       19   
4          0.0         0.0  ...  107      19      26    20       17       13   
24         0.0         0.0  ...  105      17      21    26       25       21   
26         0.0         0.0  ...  105      22      19    25       16       21   
28         0.0         0.0  ...  104      21      29    25       29       22   

    개인기  급여  left  right  
2     2  24     3      5  
4     3 

In [ ]:
data_CM_num = data_CM[['키', '몸무게', '속력', '가속력', 
       '슛 파워', '중거리 슛', '위치 선정', '짧은 패스', '시야', '크로스', '긴 패스',
       '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '대인 수비', '태클',
       '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프', '침착성',
       '개인기', '급여', 'left', 'right',
       ]]
data_CAM_num = data_CAM[['키', '몸무게', '속력', '가속력', '골 결정력',
       '슛 파워', '중거리 슛', '위치 선정', '짧은 패스', '시야', '크로스', '긴 패스', 
       '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '헤더', '몸싸움', '스태미너', 
       '적극성', '점프', '침착성', '개인기', '급여', 'left', 'right',
       ]]
data_WM_num = data_WM[['키', '몸무게', '속력', '가속력',
       '중거리 슛', '시야', '크로스', '긴 패스',
       '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도',
       '몸싸움', '스태미너', '적극성', '침착성', 
       '개인기', '급여', 'left', 'right',
       ]]

print(data_CM_num.head())
print(data_CAM_num.head())
print(data_WM_num.head())

      키  몸무게   속력  가속력  슛 파워  중거리 슛  위치 선정  짧은 패스   시야  크로스  ...  슬라이딩 태클  \
2   180   76   95   93    89     98     95    105  102   87  ...      108   
4   171   71   96   95   112    114    102    109  108  105  ...       62   
24  192   82  103  102   104    101    103    107   97   86  ...      105   
26  185   79   98   98   108    105     96    107   90   95  ...      100   
28  174   70   99  100    78     74     80    107   98   88  ...      102   

    몸싸움  스태미너  적극성   점프  침착성  개인기  급여  left  right  
2   105   108  115   89  100    2  24     3      5  
4    90   105  108  101  107    3  24     3      5  
24  113   112  110  108  105    3  26     3      5  
26  107   111  110   93  105    3  24     5      3  
28  102   109  107  103  104    3  24     3      5  

[5 rows x 31 columns]
      키  몸무게   속력  가속력  골 결정력  슛 파워  중거리 슛  위치 선정  짧은 패스   시야  ...   헤더  몸싸움  \
8   173   70   98  105     91   102    101    100     97  100  ...   84   62   
11  180   74  101  102    100    99 

In [ ]:
cos_CM = cosine_similarity(data_CM_num, data_CM_num)
cos_CM

array([[1.        , 0.98864309, 0.99834386, ..., 0.83591798, 0.97262789,
        0.99020301],
       [0.98864309, 1.        , 0.98984299, ..., 0.82842677, 0.98744528,
        0.98881968],
       [0.99834386, 0.98984299, 1.        , ..., 0.84489035, 0.97670979,
        0.99012397],
       ...,
       [0.83591798, 0.82842677, 0.84489035, ..., 1.        , 0.87919098,
        0.85949369],
       [0.97262789, 0.98744528, 0.97670979, ..., 0.87919098, 1.        ,
        0.98716097],
       [0.99020301, 0.98881968, 0.99012397, ..., 0.85949369, 0.98716097,
        1.        ]])

In [ ]:
cos_CAM = cosine_similarity(data_CAM_num, data_CAM_num)
cos_CAM

array([[1.        , 0.99572404, 0.99799551, ..., 0.97452539, 0.97975987,
        0.85236243],
       [0.99572404, 1.        , 0.99781134, ..., 0.98335326, 0.9875653 ,
        0.86577309],
       [0.99799551, 0.99781134, 1.        , ..., 0.97634036, 0.98148408,
        0.85543799],
       ...,
       [0.97452539, 0.98335326, 0.97634036, ..., 1.        , 0.99823078,
        0.9134683 ],
       [0.97975987, 0.9875653 , 0.98148408, ..., 0.99823078, 1.        ,
        0.90812674],
       [0.85236243, 0.86577309, 0.85543799, ..., 0.9134683 , 0.90812674,
        1.        ]])

In [ ]:
cos_WM = cosine_similarity(data_WM_num, data_WM_num)
cos_WM

array([[1.        , 0.99676542, 0.99900399, ..., 0.99182324, 0.98420369,
        0.98157784],
       [0.99676542, 1.        , 0.99710858, ..., 0.99295866, 0.99022058,
        0.98589521],
       [0.99900399, 0.99710858, 1.        , ..., 0.99074814, 0.98652041,
        0.98211513],
       ...,
       [0.99182324, 0.99295866, 0.99074814, ..., 1.        , 0.99135086,
        0.9922963 ],
       [0.98420369, 0.99022058, 0.98652041, ..., 0.99135086, 1.        ,
        0.99694654],
       [0.98157784, 0.98589521, 0.98211513, ..., 0.9922963 , 0.99694654,
        1.        ]])

프리킥과 페널티킥은 제외됨

#수비수

수비수 공통 중요 능력치
- 헤딩, 위치 선정, 짧은 패스, 볼 컨트롤, 태클, 슬라이딩 태클

속가, (슛파워), 짧패, 시야, 긴패스, (드리블), 민, 밸, 가로, 태클, 헤더, 반응, 몸싸, (적극성), 점프or키

센터백
- 공통 중요 능력치 외 반응 속도, 점프, 몸싸움, 대인 수비, 적극성

풀백
- 공통 중요 능력치 외 가속력, 반응 속도, 스테미너, 크로스, 드리블, 대인 수비

In [ ]:
isCB = data_B['포지션'].str.contains("CB") #중미
isWB = data_B['포지션'].str.contains("WB|LB|RB") #공미

data_CB = data_B[isCB]
data_WB = data_B[isWB]

print(data_CB.head())
print(data_WB.head())

   포지션   스탯         ID         이름  GK공격가담  GK능숙한펀칭  GK멀리던지기  GK소극적크로스수비  \
6   CB  103  101000388       솔 캠벨     0.0      0.0      0.0         0.0   
15  CB  107  101001088  알레산드로 네스타     0.0      0.0      0.0         0.0   
16  CB  108  101001109    파올로 말디니     0.0      0.0      0.0         0.0   
18  CB  104  101001116    마르셀 드사이     0.0      0.0      0.0         0.0   
19  CB  106  101001183   파비오 칸나바로     0.0      0.0      0.0         0.0   

    GK적극적크로스수비  GK침착한1:1수비  ...  침착성  GK 다이빙  GK 핸들링  GK 킥  GK 반응속도  GK 위치선정  \
6          0.0         0.0  ...   98      16      15    25       23       16   
15         0.0         0.0  ...  105      18      19    23       13       12   
16         0.0         0.0  ...  105      22      21    26       13       17   
18         0.0         0.0  ...  103      17      22    18       17       16   
19         0.0         0.0  ...   97      25      20    28       27       27   

    개인기  급여  left  right  
6     2  24     3      5  
15    2  25   

In [ ]:
data_CB_num = data_CB[['키', '몸무게', '속력', '가속력',
       '위치 선정', '짧은 패스', '시야', '긴 패스',
       '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '대인 수비', '태클',
       '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프', '침착성',
       '개인기', '급여', 'left', 'right',
       ]]      
data_WB_num = data_WB[['키', '몸무게', '속력', '가속력',
       '위치 선정', '짧은 패스', '시야', '크로스', '긴 패스',
       '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '대인 수비', '태클',
       '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프', '침착성',
       '개인기', '급여', 'left', 'right',
       ]]

print(data_CB_num.head())
print(data_WB_num.head())

      키  몸무게   속력  가속력  위치 선정  짧은 패스  시야  긴 패스  볼 컨트롤  민첩성  ...  슬라이딩 태클  몸싸움  \
6   188   89  104  100     83     94  83    85     85   88  ...      105  115   
15  187   79  101  103     68     97  97   103    100   94  ...      111  104   
16  186   77  110  107     94    102  98   104    101  105  ...      112  107   
18  185   81  100   96     86     95  88    98     96  102  ...      107  108   
19  176   73   96  103     54     89  75    86     92   98  ...      112  103   

    스태미너  적극성   점프  침착성  개인기  급여  left  right  
6     94  108  104   98    2  24     3      5  
15    96  108  102  105    2  25     3      5  
16   108  104  108  105    3  26     4      5  
18   103  108  108  103    2  24     3      5  
19    95  106  111   97    2  24     3      5  

[5 rows x 26 columns]
      키  몸무게   속력  가속력  위치 선정  짧은 패스   시야  크로스  긴 패스   커브  ...  슬라이딩 태클  몸싸움  \
3   179   79  102  100     70     98   84  107    93   86  ...      105   96   
12  168   70  112  115     98    103   80 

In [ ]:
cos_CB = cosine_similarity(data_CB_num, data_CB_num)
cos_CB

array([[1.        , 0.99710118, 0.99708203, ..., 0.9687422 , 0.97942811,
        0.98869935],
       [0.99710118, 1.        , 0.99820579, ..., 0.96527289, 0.98236615,
        0.99029426],
       [0.99708203, 0.99820579, 1.        , ..., 0.96908529, 0.98007741,
        0.98969623],
       ...,
       [0.9687422 , 0.96527289, 0.96908529, ..., 1.        , 0.97472315,
        0.9818411 ],
       [0.97942811, 0.98236615, 0.98007741, ..., 0.97472315, 1.        ,
        0.9974024 ],
       [0.98869935, 0.99029426, 0.98969623, ..., 0.9818411 , 0.9974024 ,
        1.        ]])

In [ ]:
cos_WB = cosine_similarity(data_WB_num, data_WB_num)
cos_WB

array([[1.        , 0.99570978, 0.99760274, ..., 0.94458634, 0.98515957,
        0.993711  ],
       [0.99570978, 1.        , 0.9958793 , ..., 0.94939055, 0.97232133,
        0.98961578],
       [0.99760274, 0.9958793 , 1.        , ..., 0.9493601 , 0.98137689,
        0.99311296],
       ...,
       [0.94458634, 0.94939055, 0.9493601 , ..., 1.        , 0.93888104,
        0.96483584],
       [0.98515957, 0.97232133, 0.98137689, ..., 0.93888104, 1.        ,
        0.98920554],
       [0.993711  , 0.98961578, 0.99311296, ..., 0.96483584, 0.98920554,
        1.        ]])

#골키퍼

골키퍼 중요 능력치
- 반응 속도, 점프, GK다이빙, GK핸들링, GK골킥, GK반응속도, GK위치선정


In [ ]:
data_GK_num = data_GK[['키', '몸무게',
       '위치 선정', '짧은 패스', '시야', '긴 패스',
       '민첩성', '밸런스', '반응 속도',
       '적극성', '점프', '침착성', 'GK 다이빙',
       'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정', '급여'
       ]]

print(data_GK_num.head())

      키  몸무게  위치 선정  짧은 패스  시야  긴 패스  민첩성  밸런스  반응 속도  적극성   점프  침착성  GK 다이빙  \
0   193   93     35     50  80    51   63   62    102   51   77   86      98   
9   190   87     36     43  88    28   83   69    102  102   88   88     102   
40  185   80     44     46  75    43   96   74    105   51  102   98     109   
57  184   78     44     56  43    56   96   88    103   65   97   84     104   
67  196   90     40     56  83    65   87   83     98   72   93   92     105   

    GK 핸들링  GK 킥  GK 반응속도  GK 위치선정  급여  
0      105   106      101      103  20  
9      106   101      101      105  21  
40     108    98      109      109  22  
57     103    95      102       99  20  
67     107    98      110      108  22  
